### 코랩에서 한글 사용하기
- 한글 글꼴 설치 => [런타임]-[런타임 다시 시작] 클릭


### 서울시 유가정보(11개 구) 분서

#### [미션1] 서울시 유가정보 가져와 1개로 합치기
1. "/content" 폴더에 "data" 폴더 생성후 파이 업로드 
2. data 폴더에 있는 데이터 읽어와 oildf에 모두 저장
3. 데이터는 concat()을 이용해 데이터의 아래에 추가
4. encoding: cp949 또는 utf8이 같이 있어 에러 발생
        try: ~ except: ~ 로 해결



- data 폴더에 있는 파일명/하위폴더명 가져오기: os모듈
- for문을 이용 첫번째 파일부터 순차적으로 읽어와 oildf에 concat 진행
- 파일을 읽을시 encoding 에러
        try: 
            utf-8 형식 읽어오기
        except:
            cp949 형식 읽어오기
- 빈 DataFrame 변수 생성
        oildf=pd.DataFrame()

In [1]:
import pandas as pd
import os

fnames=os.listdir("/content/data")

oildf= pd.DataFrame()
for fname in fnames:
    try:
        df0=pd.read_csv("/content/data/" + fname, header=2)
    except:
        df0=pd.read_csv("/content/data/" + fname, header=2, encoding="cp949")

    oildf=pd.concat([oildf, df0])  # 옵션: ignore_index=True : index 재설정

del df0

oildf.reset_index(drop=True, inplace=True)  # 

display(oildf.head(3))
oildf.info()

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,서울특별시,현대오일뱅크(주)직영 팔레스주유소,서울 서초구 사평대로 136 (반포동),현대오일뱅크,02-720-0020,N,2135,1975.0,1913.0,-
1,서울특별시,신반포주유소,서울 서초구 사평대로 94 (반포동),SK에너지,02-533-5151,N,2138,1978.0,1918.0,1300
2,서울특별시,만남의광장주유소,서울 서초구 양재대로12길 73-71,알뜰(ex),02-573-7430,Y,-,1989.0,1949.0,-


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      225 non-null    object
 1   상호      225 non-null    object
 2   주소      225 non-null    object
 3   상표      225 non-null    object
 4   전화번호    225 non-null    object
 5   셀프여부    225 non-null    object
 6   고급휘발유   203 non-null    object
 7   휘발유     220 non-null    object
 8   경유      221 non-null    object
 9   실내등유    189 non-null    object
dtypes: object(10)
memory usage: 17.7+ KB


#### [미션2] 데이터 전처리
- "구" 필드 생성하기
- 데이터 출력 필드: 상호, 상표, 구, 세프여부, 휘발유, 경우 만 출력
- NaN 값 평균 값으로 변경하기
- "휘발유", "경유": float64로 변경
- "휘발유", "경유" 데이터의 "-"를 평균값으로 수정

필드별 NaN 객수 확인

In [13]:
oildf.isna().sum()

지역        0
상호        0
주소        0
상표        0
전화번호      0
셀프여부      0
고급휘발유    22
휘발유       5
경유        4
실내등유     36
dtype: int64

구 필드 생성하기

In [ ]:
## loc[] 함수를 이용해 주소에서 구 추출하기
gutxt = []
for i in oildf.index:
    gutxt.append( oildf.loc[i, '주소'].split(" ")[1] )

In [38]:
# 한줄쓰기로 표현
gutxt=[ oildf.loc[i, ['주소']].split(" ")[1] for i in oildf.index ]

AttributeError: ignored

In [16]:
# iloc[] 함수를 이용해 주소에서 구 추출하기
gutxt=[ oildf.iloc[i, 2].split(" ")[1] for i in range(len(oildf)) ]

In [17]:
# "구" 필드 추가
oildf["구"]=gutxt

상호', '상표', "구", '셀프여부', '휘발유', '경유' 필드만 출력

In [18]:
# 필드명으로 DataFrame 생성하기
oildf1=oildf[['상호','상표',"구",'셀프여부', '휘발유', '경유']]

In [19]:
# iloc를 이용한 DataFrame 생성하기
oildf2=oildf.iloc[:, [1, 3,-1, 5, 7, 8]]
oildf2.dtypes

상호      object
상표      object
구       object
셀프여부    object
휘발유     object
경유      object
dtype: object

NaN / "-" 값 0으로 변경하기

In [21]:
# NaN값 0으로 변경
oildf2=oildf2.fillna(0)

In [22]:
# 휘발유/경우 데이터에서 "-" 값만 찾아 0으로 변경
oildf2.loc[oildf2.휘발유=="-",'휘발유']=0
oildf2.loc[oildf2["경유"]=="-",'경유']=0

In [23]:
# 고유값 확인하기
oildf2['휘발유'].unique()

array([1975.0, 1978.0, 1989.0, 1998.0, 1999.0, 2008.0, 2010.0, 2020.0,
       2022.0, 2025.0, 2028.0, 2029.0, 2031.0, 2035.0, 2047.0, 2048.0,
       2062.0, 2071.0, 2073.0, 2074.0, 2088.0, 2089.0, 2095.0, 2138.0,
       2198.0, 2238.0, 2298.0, 2477.0, 1987.0, 1988.0, 1997.0, 2039.0,
       2059.0, 2069.0, 2199.0, 2299.0, 1993.0, 0, 2018.0, 2065.0, 2098.0,
       2172.0, '1949', '1954', '1973', '1985', '2014', '2018', '2025',
       '2027', '2048', '2068', 1955.0, 2038.0, 2040.0, 2045.0, 2052.0,
       2055.0, 2079.0, 2097.0, 2108.0, 2158.0, 2194.0, 2228.0, 2239.0,
       2243.0, 2288.0, 2296.0, 2329.0, 2380.0, 2393.0, 2394.0, 2398.0,
       2545.0, 1926, 1979, 1986, 1995, 1996, 2099, 2177, '1945', '1995',
       '1998', '1999', '2007', '2011', '2028', '2039', '2045', '2053',
       '2065', '2069', '2085', 1968, 2009, 2015, 2090, '2089', '2119',
       '2120', '2345', '2378', '2395', '2398', 1985, 2023, 2027, 2148],
      dtype=object)

데이터형(data type) 변경하기

In [24]:
# 휘발유/경우 실수형 자료로 변경하기
oildf2=oildf2.astype({"휘발유":float, "경유":float})    # astype : pandas 에서만 사용가능
oildf2.dtypes  # 데이터 타입 확인하기

상호       object
상표       object
구        object
셀프여부     object
휘발유     float64
경유      float64
dtype: object

휘발유/경유가 0인 값 평균으로 변경하기

In [25]:
# 휘발유/경유 데이터에서 0인 값을 평균값으로 변경하기
oildf2.loc[oildf2.휘발유==0,'휘발유']=oildf2["휘발유"].mean()
oildf2.loc[oildf2["경유"]==0,'경유']=oildf2["경유"].mean()

In [27]:
# 0인 값이 인는지 확인
oildf2.loc[oildf2["경유"]==0, :]
oildf2.loc[oildf2.휘발유==0, :]

,상호,상표,구,셀프여부,휘발유,경유


#### pandas 모듈을 이용한 데이터 분석(기본)
- groupby: 기준이 되는 필드의 값을 기준으로 묶어서 통계 또는 집계 계산

In [29]:
# oildf2에서 휘발유 최고가 검색

oildf2["휘발유"].max()
oildf2[oildf2["휘발유"] == oildf2["휘발유"].max()]

,상호,상표,구,셀프여부,휘발유,경유
33,제이제이주유소,현대오일뱅크,강남구,N,2545.0,2335.0


In [32]:
# "구" 필드를 기준으로 평균 값 계산

guGroup = oildf2.groupby(by="구").mean()       # group 항목이 index로 설정
guGroup.head()

휘발유    2216.521481
경유     2099.130864
dtype: float64

In [37]:
# 강북구 데이터만 출력

guGroup.loc[["강북구", "양천구"]]

,휘발유,경유
구,,
강북구,2001.230769,1912.153846
양천구,2001.760000,1929.840000


In [39]:
guGroup.index

Index(['강남구', '강동구', '강북구', '노원구', '도봉구', '마포구', '서초구', '송파구', '양천구', '은평구',
       '종로구'],
      dtype='object', name='구')

In [46]:
# 범위지정 출력

guGroup.loc[['강북구', '노원구', '도봉구', '마포구', '서초구']]
guGroup.loc['강북구' : '서초구', "휘발유"]

구
강북구    2001.230769
노원구    2035.933333
도봉구    2016.927407
마포구    2071.750000
서초구    2068.696970
Name: 휘발유, dtype: float64

- 구별 주유소 갯수 확인
- 구별 / 상표별 주유소 갯수

In [49]:
oildf2.groupby("구")["상표"].count()

구
강남구    37
강동구    14
강북구    13
노원구    15
도봉구    18
마포구    12
서초구    33
송파구    32
양천구    25
은평구    17
종로구     9
Name: 상표, dtype: int64

In [ ]:
oildf2.groupby(["구", "상표"])["휘발유", "경유"].count()      # 구별 / 상표별 주유소 갯수

In [55]:
oildf2.groupby("구").get_group("노원구")      # group 안에서 원하는 데이터 확인

,상호,상표,구,셀프여부,휘발유,경유
0,한신주유소,SK에너지,노원구,Y,1968.0,1838.0
1,삼육주유소,SK에너지,노원구,Y,1975.0,1845.0
2,태릉솔밭주유소,S-OIL,노원구,Y,1995.0,1895.0
3,화랑대주유소,현대오일뱅크,노원구,Y,1995.0,1895.0
4,이엠석유(주)배꽃나라주유소,GS칼텍스,노원구,Y,2009.0,1909.0
5,현대오일뱅크㈜직영 하계주유소,현대오일뱅크,노원구,N,2015.0,1925.0
6,월계주유소,GS칼텍스,노원구,Y,2015.0,1915.0
7,SK에너지㈜직영 상계주유소,SK에너지,노원구,N,2025.0,1965.0
8,남선석유(주)구도일주유소 불암,S-OIL,노원구,Y,2025.0,1965.0
9,노원주유소,GS칼텍스,노원구,N,2038.0,1949.0


In [ ]:
oildf2.groupby(["구", "상표"], as_index=False)["휘발유"].count()      # as_index=False 옵션 사항으로 자료형으로 만들 때 활용

- DataFrame 데이터 저장
  * df.to_csv("경로 / 파일명.csv, index = None, encoding = None
  * index = True(기본값 / index값 저장), False(index값 저장 생략)
  * encoding = 기본 "utf8"로 필요시 원하는 encoding 형식 선택 ex) encoding="cp949"

In [57]:
# oildf2 csv 파일로 저장하기

oildf2.to_csv("서울시 유류정보_0314.csv", index=False, encoding="cp949")

# 파일 다운로드 (colab에서 저장한 파일)

from google.colab import files
files.download("서울시 유류정보_0314.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>